In [31]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from datetime import datetime

In [32]:
# Load both CSV files
bottom_data = pd.read_csv('location 2 bottom.csv')
surface_data = pd.read_csv('location 2 surface.csv')



In [33]:
# Print column names to verify
print("Bottom Data Columns:", bottom_data.columns.tolist())
print("Surface Data Columns:", surface_data.columns.tolist())

Bottom Data Columns: ['Year/Month', 'Temp', 'pH', 'TDS', 'Conductivity', 'Turbidity', 'DO', 'BOD', 'COD', 'Hardness', 'Alkine', 'NO3', 'PO4', 'SO2']
Surface Data Columns: ['Year/Month', 'Temp', 'pH', 'TDS', 'Conductivity', 'Turbidity', 'DO', 'BDO', 'CDO', 'Hardnees', 'Alkanity', 'NO3  as N', 'Total PO43', 'SO42-']


In [34]:
# Cell 2: Preprocess Data
def preprocess_data(df):
    # Make a copy of the dataframe
    df = df.copy()
    
    # Convert Year/Month to datetime
    df['Date'] = pd.to_datetime(df['Year/Month'], format='%Y/%m')
    
    # Extract month as a cyclical feature
    df['Month_sin'] = np.sin(2 * np.pi * df['Date'].dt.month/12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Date'].dt.month/12)
    
    # Extract year as a numerical feature
    df['Year'] = df['Date'].dt.year - df['Date'].dt.year.min()
    
    # Handle different column names in bottom and surface data
    # For NO3
    if 'NO3' in df.columns:
        df['NO3_cleaned'] = df['NO3']
    elif 'NO3  as N' in df.columns:
        df['NO3_cleaned'] = df['NO3  as N']
        
    # For SO4
    if 'SO2' in df.columns:
        df['SO4_cleaned'] = df['SO2']
    elif 'SO42-' in df.columns:
        df['SO4_cleaned'] = df['SO42-']
        
    # Ensure pH column is consistent
    df['pH_cleaned'] = df['pH']
    
    # Fill any NaN values with the mean of the column
    df['NO3_cleaned'] = df['NO3_cleaned'].fillna(df['NO3_cleaned'].mean())
    df['SO4_cleaned'] = df['SO4_cleaned'].fillna(df['SO4_cleaned'].mean())
    df['pH_cleaned'] = df['pH_cleaned'].fillna(df['pH_cleaned'].mean())
    
    return df

# Preprocess both datasets
bottom_processed = preprocess_data(bottom_data)
surface_processed = preprocess_data(surface_data)

# Combine datasets
combined_data = pd.concat([bottom_processed, surface_processed], axis=0)

# Select input features and target variables
input_features = ['Temp', 'Month_sin', 'Month_cos', 'Year']
target_features = ['NO3_cleaned', 'SO4_cleaned', 'pH_cleaned']

# Create input and output arrays
X = combined_data[input_features].values
y = combined_data[target_features].values

# Normalize the data
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_normalized = X_scaler.fit_transform(X)
y_normalized = y_scaler.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y_normalized, test_size=0.2, random_state=42
)


In [35]:
# Cell 3: Create Neural Network Model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(3)  # Output layer for NO3, SO4, and pH
    ])
    return model

model = create_model()


In [36]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Mean Absolute Error: {test_mae:.4f}")

# Function to make predictions
def predict_water_quality(temperature, date_str):
    # Convert date string to features
    date = datetime.strptime(date_str, '%Y/%m')
    month_sin = np.sin(2 * np.pi * date.month/12)
    month_cos = np.cos(2 * np.pi * date.month/12)
    year = date.year - bottom_processed['Date'].dt.year.min()
    
    # Create input array
    input_data = np.array([[temperature, month_sin, month_cos, year]])
    
    # Normalize input
    input_normalized = X_scaler.transform(input_data)
    
    # Make prediction
    prediction_normalized = model.predict(input_normalized)
    
    # Denormalize prediction
    prediction = y_scaler.inverse_transform(prediction_normalized)
    
    return {
        'NO3': float(prediction[0][0]),
        'SO4': float(prediction[0][1]),
        'pH': float(prediction[0][2])
    }

# Example usage:
example_prediction = predict_water_quality(25.0, '2024/06')
print("\nExample Prediction:")
print(example_prediction)

# Print some actual data ranges for verification
print("\nData Ranges:")
print(f"NO3 range: {combined_data['NO3_cleaned'].min():.2f} to {combined_data['NO3_cleaned'].max():.2f}")
print(f"SO4 range: {combined_data['SO4_cleaned'].min():.2f} to {combined_data['SO4_cleaned'].max():.2f}")
print(f"pH range: {combined_data['pH_cleaned'].min():.2f} to {combined_data['pH_cleaned'].max():.2f}")

Epoch 1/100
17/17 [==============================] - 1s 13ms/step - loss: 0.0803 - mae: 0.2136 - val_loss: 0.0297 - val_mae: 0.1305
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0400 - mae: 0.1499 - val_loss: 0.0257 - val_mae: 0.1213
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0353 - mae: 0.1404 - val_loss: 0.0248 - val_mae: 0.1212
Epoch 4/100
17/17 [==============================] - 0s 3ms/step - loss: 0.0312 - mae: 0.1334 - val_loss: 0.0238 - val_mae: 0.1183
Epoch 5/100
17/17 [==============================] - 0s 5ms/step - loss: 0.0307 - mae: 0.1296 - val_loss: 0.0234 - val_mae: 0.1171
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0300 - mae: 0.1271 - val_loss: 0.0230 - val_mae: 0.1166
Epoch 7/100
17/17 [==============================] - 0s 3ms/step - loss: 0.0280 - mae: 0.1261 - val_loss: 0.0226 - val_mae: 0.1161
Epoch 8/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0280 - 